### Тематика последующего обращения (NN)

In [ ]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('dialogs_2.csv')

In [35]:
data.head()

,appeal_id,datetime,client_id,user_id,msg_type,channel_id,channel_name,channel_type,answer_time,text
0,323874,2088-08-08 16:44:02+03,NaN,19670.0,0,73,Webdealer,Веб-чат,12.0,Привет.))
1,323870,2088-08-08 16:44:01+03,NaN,19671.0,0,73,Webdealer,Веб-чат,NaN,Все подписки отключены. Как это делать самосто...
2,323861,2088-08-08 16:44:01+03,NaN,19418.0,0,133,Нижний Новгород B2C,Веб-чат,NaN,Вы забыли пароль и хотите войти в кабинет?
3,323850,2088-08-08 16:44:01+03,NaN,19578.0,0,2,Viber,Viber,54.0,Можем отправить заявку на подключение новых но...
4,322276,2088-08-08 16:44:01+03,NaN,NaN,2,0,NaN,NaN,NaN,Обращение закрыто в связи с неактивностью поль...


In [3]:
data_one_day = data.dropna(subset=['client_id']).copy()

In [4]:
data_one_day = data_one_day[~data_one_day['client_id'].isnull()]

In [5]:
data_one_day = data_one_day.sort_values(by=['client_id'])

In [6]:
data_one_day = data_one_day[data_one_day['client_id'] > 2000]

In [7]:
data_one_day = data_one_day[~data_one_day['text'].isnull()]

In [8]:
data_one_day.iloc[203]

appeal_id                                                  257043
datetime                                   2088-07-11 10:44:30+03
client_id                                                  357140
user_id                                                       NaN
msg_type                                                        0
channel_id                                                      2
channel_name                                                Viber
channel_type                                                Viber
answer_time                                                   NaN
text            Привет . В письме детализации звонков , нету ф...
Name: 1145599, dtype: object

In [9]:
len(np.unique(data_one_day['client_id']))

130515

In [ ]:
'интернет': ['скорость', 'интернет', 'сайт', 'знач', 'антен', 'скорость', 'тормозит', 'грузит', 'онлайн', '4g', '3g', '2g'],
'услуги': ['услуг', 'снял', 'отключ', 'подпис'],
'тариф': ['тариф', 'стоимост', 'плат'],

### Препроцессинг

In [11]:
alphabet = {i:k for k, i in enumerate(' абвгдежзийклмнопрстуфхцчшщъыьэюя')}

In [12]:
topic = []
boc = []
for i in range(len(data_one_day)):
    flag = 1
    text_poverka = data_one_day.iloc[i]['text'].lower()
    tmp_boc = np.zeros(50)
    for j in range(min(50, len(text_poverka))):
        if text_poverka[j] in alphabet:
            tmp_boc[j] = alphabet[text_poverka[j]]
    boc.append(tmp_boc)
    for el in ['скорость', 'интернет', 'сайт', 'знач', 'антен', 'скорость', 'тормозит', 'грузит', 'онлайн', '4g', '3g', '2g']:
        if el in text_poverka:
            topic.append(1)
            flag = 0
            break
    if flag:
        for el in ['услуг', 'снял', 'отключ', 'подпис']:
            if el in text_poverka:
                topic.append(2)
                flag = 0
                break
    if flag:
        for el in ['тариф', 'стоимост', 'плат']:
            if el in text_poverka:
                topic.append(3)
                flag = 0
                break
    if flag:
        topic.append(0)

KeyboardInterrupt: 

In [14]:
data_topic = data_one_day.iloc[:len(topic)].copy()

In [15]:
data_topic['tipic'] = topic
data_topic['boc'] = boc

In [16]:
data_topic.head()

,appeal_id,datetime,client_id,user_id,msg_type,channel_id,channel_name,channel_type,answer_time,text,tipic,boc
3374548,131444,2088-05-03 16:59:42+03,357128.0,NaN,0,2,Viber,Viber,NaN,Маленькая скорость,1,"[13.0, 1.0, 12.0, 6.0, 14.0, 29.0, 11.0, 1.0, ..."
3374547,131444,2088-05-03 16:59:51+03,357128.0,NaN,0,2,Viber,Viber,NaN,xxx xxx xx xx,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3374546,131444,2088-05-03 16:59:14+03,357128.0,NaN,0,2,Viber,Viber,NaN,Интернет плохо пашет,1,"[9.0, 14.0, 19.0, 6.0, 17.0, 14.0, 6.0, 19.0, ..."
3368876,131678,2088-05-04 09:41:13+03,357128.0,NaN,0,2,Viber,Viber,NaN,Интернет везде плохой,1,"[9.0, 14.0, 19.0, 6.0, 17.0, 14.0, 6.0, 19.0, ..."
3366410,131678,2088-05-04 13:18:04+03,357128.0,NaN,0,2,Viber,Viber,NaN,Значек Е. Две антенки,1,"[8.0, 14.0, 1.0, 24.0, 6.0, 11.0, 0.0, 6.0, 0...."


In [17]:
train = []
label = []
for client_id in np.unique(data_topic['client_id']):
    tmp = data_topic[data_topic['client_id'] == client_id]
    train.append(np.array(tmp['boc'][:-1]))
    label.append(np.array(tmp['tipic'][1:]))

In [27]:
train_x = []
for i in range(len(train)):
    train_x.extend(train[i])
train_x = np.array(train_x)

In [31]:
train_x[:5]

array([[13.,  1., 12.,  6., 14., 29., 11.,  1., 32.,  0., 18., 11., 15.,
        17., 15., 18., 19., 29.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 9., 14., 19.,  6., 17., 14.,  6., 19.,  0., 16., 12., 15., 22.,
        15.,  0., 16.,  1., 25.,  6., 19.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 9., 14., 19.,  6., 17., 14.,  6., 19.,  0.,  3.,  6.,  8.,  5.,
         6.,  0., 16., 12., 15., 22., 15., 10.,  0.,  0.,  0.,  0.,  0.,
     

In [32]:
label_x = []
for i in range(len(label)):
    label_x.extend(label[i])
label_x = np.array(label_x)

In [33]:
label_x[:5]

array([0, 1, 1, 1, 0])

In [34]:
len_test = int(len(train_x) / 5)
test = train_x[-len_test:]
test_label = label_x[-len_test:]
train = train_x[:-len_test]
label = label_x[:-len_test]

In [35]:
from keras.models import Sequential
from keras.layers import Dense

/root/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/root/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/root/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/root/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [47]:
from keras.utils import to_categorical
y_binary = to_categorical(label)

In [49]:
y_binary[1]

array([0., 1., 0., 0.], dtype=float32)

### Модель

In [55]:
model = Sequential()
model.add(Dense(units=15, activation='relu'))
model.add(Dense(units=4, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

### Обучение модели

In [56]:
model.fit(train, y_binary, epochs=5, batch_size=32)

Epoch 1/5
348500/348500 [==============================] - 32s 93us/step - loss: 0.6878 - acc: 0.8191
Epoch 2/5
348500/348500 [==============================] - 31s 90us/step - loss: 0.6645 - acc: 0.8202
Epoch 3/5
348500/348500 [==============================] - 30s 85us/step - loss: 0.6641 - acc: 0.8202
Epoch 4/5
348500/348500 [==============================] - 31s 89us/step - loss: 0.6639 - acc: 0.8202
Epoch 5/5
348500/348500 [==============================] - 32s 92us/step - loss: 0.6638 - acc: 0.8202


In [57]:
y_test = to_categorical(test_label)

In [58]:
loss_and_metrics = model.evaluate(test, y_test, batch_size=32)

2000/2000 [==============================] - 0s 117us/step


### Качество модели

In [59]:
loss_and_metrics

[0.6748291068077087, 0.816]